In [2]:
from jupyter_plotly_dash import JupyterDash
import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "myAacPass"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
            style={'display': 'flex', 'width': '100%', 'height': '100%'},
            children=[
                html.Div(html.B(html.H1('Scott Vanderwilt SNHU CS-340 Dashboard'))),
                html.Div(html.A([
                    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                    style={'height': 100, 'width': 100,},)]),)
    ]),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-radio',
            options = [
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain Rescue', 'value': 'MR'},
                {'label': 'Disaster Rescue', 'value': 'DR'},
                {'label': 'Reset', 'value': 'RS'},
            ],
            value = 'RS'
        )
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        sort_action="native",
        sort_mode="multi",
        page_current=0,
        page_size=10,
        selected_rows=[0],
        column_selectable=False,
        row_selectable='single',
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
         style={'display' : 'flex', 'width': '100%', 'height': 'auto'},
             children=[
        html.B(
            dcc.Graph(id='graph-id'),
            style={'flex': '1', 'border': '2px solid yellow'}
            #className='col s12 m6',

            ),
        html.A(
            id='map-id',
            style={'flex': '1', 'border': '2px solid yellow'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-radio', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if value == 'WR':
            df = get_data(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
                          'Intact Female', 26, 156)
        
        elif value == 'MR':
            df = get_data(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
                          'Intact Male', 26, 156)

        elif value == 'DR':
            df = get_data(['Doberman Pinsher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
                          'Intact Male', 20, 300)
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
    
# helper function to get data from parameters
def get_data(breeds, sex, min_age, max_age):
    return pd.DataFrame.from_records(shelter.read(
    {
        'animal_type':'Dog',
        'breed':{'$in':breeds},
        'sex_upon_outcome': sex,
        'age_upon_outcome_in_weeks':{'$gte':min_age, '$lte': max_age},
    }))




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', 'figure'),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    fig=px.pie(dff, names='breed', title="Breeds Available")
    fig.update_traces(showlegend=False)
    return fig

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, selected_row):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75, -97.48] [30.25, -97.75]
    return [
        dl.Map(style={'width': '100%', 'height': '100%'}, center=[30.25, -97.75], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # marker with tool tip and popup
            dl.Marker(position=[30.25, -97.75], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row,9])
                ])
            ])
        ])
    ]


app